In [1]:
from broadcaster_nets_2_3 import *

from BCMPLayer import *

from torch_geometric.datasets import Planetoid

import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

import torch.nn as N

import itertools

import csv
from time import perf_counter


from layer2Test import netTest2layer
from netsGenerator import *
from time import perf_counter
import sys
import copy

import faiss
import faiss.contrib.torch_utils


import time
import torch

# import os
# os.environ['CUDA_PATH'] = '/modules/apps/cuda/11.3.1' 
# import pykeops
# from pykeops.torch import LazyTensor
# # pykeops.clean_pykeops() 
# pykeops.test_torch_bindings()

from guppy import hpy
from matplotlib import pyplot as plt

print(torch.cuda.is_available())
!nvidia-smi


True
Tue Mar  7 19:42:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1A:00.0 Off |                  N/A |
| 23%   32C    P8     9W / 250W |      4MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
# import gc
# for obj in gc.get_objects():
#     try:
#         if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#             print(type(obj), obj.size())
#     except:
#         pass
# print(torch.cuda.memory_summary(device=None, abbreviated=False))

/work/ssahibul_umass_edu/.conda/envs/bc113/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/home/ssahibul_umass_edu/.local/lib/python3.8/site-packages/torch_geometric/graphgym/config.py:19: UserWarning: Could not define global config object. Please install 'yacs' for using the GraphGym experiment manager via 'pip install yacs'.
  warnings.warn("Could not define global config object. Please install "


<class 'torch.Tensor'> torch.Size([19])
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool 

/home/ssahibul_umass_edu/.local/lib/python3.8/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/ssahibul_umass_edu/.local/lib/python3.8/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cora = dataset[0].to(device)

res = faiss.StandardGpuResources()
bc_assigments, bc_features, counter = kmeansFaissGood(cora.x,k=7,device = device, maxIters=200, centroids=None, res = res)


In [4]:
from typing import Optional

import torch

import torch_geometric.utils 

class BCMPLayer:
    def __init__(self,dimWX,dimWZ,dimwalpha,device):
        #
        self.WX = torch.nn.parameter.Parameter(torch.rand(size=dimWX,device=device),requires_grad=True) #how init?
        self.WZ =  torch.nn.parameter.Parameter(torch.rand(size=dimWZ,device=device),requires_grad=True) #how init?
        self.Walpha =  torch.nn.parameter.Parameter(torch.rand(size=dimwalpha,device=device),requires_grad=True) #how init?
        self.fuse = lambda Xold,Xprim,Zprime,Zalpha : torch.mean(torch.stack((Xold,Xprim,Zprime,Zalpha),dim=2),dim=2)
        self.device = device
        
    def forward(self,x,edge_index,bc_feature,bc_assigment):
        A = torch.sparse_coo_tensor(edge_index,torch.ones(edge_index.size(dim=1),device=self.device),(x.size(dim=0),x.size(dim=0)),device=self.device)
        Z = bc_feature
        bc_assigment_edges = torch.vstack((torch.arange(0,(bc_assigment.size(dim=0)),device=self.device),bc_assigment)) # non symetric
        bc_assigment_edges_2 =  torch.vstack((bc_assigment, torch.arange(0,(bc_assigment.size(dim=0)),device=self.device))) # reverse symetric
        bc_assigment_edges = torch.cat((bc_assigment_edges,bc_assigment_edges_2),dim=1) # symetric
        print(bc_assigment_edges.shape)
        B =  torch.sparse_coo_tensor(bc_assigment_edges,torch.ones(bc_assigment_edges.size(dim=1),device=self.device),(x.size(dim=0),bc_feature.size(dim=0)),device=self.device) #rectangle
        #square b                             
        # bc_assigment_edges = torch.tensor([torch.range(0,(bc_feature.size(dim=0))-1,device=self.device),torch.add(bc_assigment+x.size(dim=0))]device=self.device) # non symetric
        # # bc_assigment_edges_2 = torch.tensor([torch.add(bc_assigment+x.size(dim=0)),torch.range(0,(bc_feature.size(dim=0))-1,device=self.device)],device=self.device) # reverse symetric
        # # bc_assigment_edges = torch.cat((bc_assigment_edges,bc_assigment_edges_2),dim=1) # symetric
        # b =  torch.sparse_coo_tensor(bc_assigment_edges,torch.ones(bc_assigment_edges.size(dim=0)),(x.size(dim=0)+bc_feature.size(dim=0),x.size(dim=0)+bc_feature.size(dim=0),device=self.device) #square
                               
        Xprime = x@self.WX
        Xold = Xprime
        Zprime = Z@self.WZ
        Zalpha = Z@self.Walpha
        Aselfloop = torch.eye(A.size(dim=0),device=self.device)+A#no longer sparse
        Bselfloop = torch.eye(n=B.size(dim=0),m=B.size(dim=1),device=self.device)+B#no longer sparse
        print(Aselfloop.is_sparse)
        D = torch.diag(torch.sum(Aselfloop,dim=0)) # include self-loop
        Drow = torch.diag(torch.sum(Bselfloop,dim=1)) # include self-loop
        Dcol = torch.diag(torch.sum(torch.transpose(Bselfloop,0,1),dim=1)) # include self-loop 
        Ahat = torch.pow(D,.5)@(Aselfloop)@torch.pow(D,.5)
        Bhat = torch.pow(Drow,.5)@(Bselfloop)@torch.pow(Dcol,.5)
        print(Ahat.is_sparse)
        print(Xprime.is_sparse)
        Ahat = Ahat.to_sparse()
        Bhat = Bhat.to_sparse()
        print(Ahat.is_sparse)
        # Xprime = torch_geometric.utils.spmm(Ahat.to_sparse(),Xprime,"sum")
        # Zprime = torch_geometric.utils.spmm(Bhat.to_sparse(),Zprime,"sum")
        # Zalpha = torch_geometric.utils.spmm(Ahat.to_sparse(),spmm(Bhat,Zalpha,"sum"),"sum")
        Xprime = torch.sparse.mm(Ahat,Xprime)
        Zprime = torch.sparse.mm(Bhat,Zprime)
        Zalpha = torch.sparse.mm(Ahat,torch.sparse.mm(Bhat,Zalpha))
        return self.fuse(Xold,Xprime,Zprime,Zalpha)

In [5]:
layer = BCMPLayer((cora.x.size(dim=1),64),(cora.x.size(dim=1),64),(cora.x.size(dim=1),64),device)
print(layer.forward(cora.x,cora.edge_index,bc_features,bc_assigments))

torch.Size([2, 5416])


RuntimeError: size is inconsistent with indices: for dim 1, size is 7 but found index 2707

In [9]:
WX = torch.nn.parameter.Parameter(torch.rand(size=(2708,1433),device=device),requires_grad=True)

In [10]:
print(WX)

Parameter containing:
tensor([[0.0570, 0.4245, 0.9248,  ..., 0.2860, 0.0191, 0.8456],
        [0.1581, 0.4196, 0.8478,  ..., 0.0681, 0.3777, 0.7560],
        [0.8978, 0.3547, 0.6222,  ..., 0.3651, 0.6225, 0.3101],
        ...,
        [0.8934, 0.0462, 0.3747,  ..., 0.1905, 0.5473, 0.3685],
        [0.7146, 0.5697, 0.1196,  ..., 0.7565, 0.9123, 0.8578],
        [0.8174, 0.1207, 0.7471,  ..., 0.2825, 0.0780, 0.7247]],
       device='cuda:0', requires_grad=True)
